# Registering new content

Shows how to register a new Investigation and Study and link them together

In [ ]:
import requests
import json
import string

import getpass

In [ ]:
base_url = 'https://sandbox9.fairdomhub.org'

We also provide the content type for the headers, this lets SEEK know the content will be of the type *application/vnd.api+json*

Also need to authenticate as a registered user to be able to create content.

In [ ]:
headers = {"Content-type": "application/vnd.api+json",
           "Accept": "application/vnd.api+json",
           "Accept-Charset": "ISO-8859-1"}

session = requests.Session()
session.headers.update(headers)
session.auth = (input('Username:'), getpass.getpass('Password'))

In [ ]:
containing_project_id = 8

Create a Hash that provides the necessary attributes for a new Investigation.

To associate the investigation, the relationships block needs populating.

The structure of the JSON used to create or update items, corresponds the resource JSON produced through a GET.

In [ ]:
investigation = {}
investigation['data'] = {}
investigation['data']['type'] = 'investigations'
investigation['data']['attributes'] = {}
investigation['data']['attributes']['title'] = 'my lovely investigation'
investigation['data']['attributes']['description'] = 'the description of my lovely investigation'
investigation['data']['relationships'] = {}
investigation['data']['relationships']['projects'] = {}
investigation['data']['relationships']['projects']['data'] = [{'id' : containing_project_id, 'type' : 'projects'}]

To create an item we do a POST to the root of that resources path. So for investigation it is https://sandbox9.fairdomhub.org/investigations

There is no ID yet, as we don't know it until it is created.

If the item is sucessfully created, the response contains the JSON of the newly created item. 

If not provided, the policy adopts the default - in this case private.

In [ ]:
r = session.post(base_url + '/investigations', json=investigation)
r.raise_for_status()
populated_investigation = r.json()
populated_investigation

This is the ID of the investigation which will be used to link to the Study

In [ ]:
investigation_id = populated_investigation['data']['id']
investigation_id

The content for the Study. The relationships contains the link to the Investigation using the *investigation_id* determined earlier

In this case a policy block has been provided, which makes the Study public, and Person 1 has been given manage rights.

In [ ]:
study = {}
study['data'] = {}
study['data']['type'] = 'studies'
study['data']['attributes'] = {}
study['data']['attributes']['title'] = 'my lovely study'
study['data']['attributes']['description'] = 'the description of lovely study'
study['data']['attributes']['policy'] = {'access':'view', 'permissions': [{'resource': {'id': '1','type': 'people'},'access': 'manage'}]}
study['data']['relationships'] = {}
study['data']['relationships']['investigation'] = {}
study['data']['relationships']['investigation']['data'] = {'id' : investigation_id, 'type' : 'investigations'}

In [ ]:
r = session.post(base_url + '/studies', json=study)
r.raise_for_status()
populated_study = r.json()
populated_study

The id of the created Study, to link the Assay to.

In [ ]:
study_id = populated_study['data']['id']

Now create the Assay. The assay has some additional fields:



*   assay_class - the class of the Assay. The key can be either:
  * EXP - an experimental assay
  * MODEL - a modelling analysis
*   assay_type - the Assay Type from the [JERM Ontology](https://www.jermontology.org)
*   technology_type - the Technology Type from the [JERM Ontology](https://www.jermontology.org)



In [ ]:
assay = {}
assay['data'] = {}
assay['data']['type'] = 'assays'
assay['data']['attributes'] = {}
assay['data']['attributes']['title'] = 'my lovely assay'
assay['data']['attributes']['description'] = 'the description of lovely assay'
assay['data']['attributes']['policy'] = {'access':'view', 'permissions': []}
assay['data']['attributes']['assay_class'] = {'key' : 'EXP'}
assay['data']['attributes']['assay_type'] = {'uri' : 'http://jermontology.org/ontology/JERMOntology#Metabolomics'}
assay['data']['attributes']['technology_type'] = {'uri' : 'http://jermontology.org/ontology/JERMOntology#Electrophoresis'}
assay['data']['relationships'] = {}
assay['data']['relationships']['study'] = {}
assay['data']['relationships']['study']['data'] = {'id' : study_id, 'type' : 'studies'}

Now create the Assay in the same with, with a POST

In [ ]:
r = session.post(base_url + '/assays', json=assay)
r.raise_for_status()
populated_assay = r.json()
populated_assay

In [ ]:
print("Assay ID: " + populated_assay['data']['id'])

Here we clean up and delete the created items. 

Items are deleted by sending a http DELETE to the resource url. They need to be deleted in reverse order else they will fail.

In [ ]:


assay_url = populated_assay['data']['links']['self']
study_url = populated_study['data']['links']['self']
investigation_url = populated_investigation['data']['links']['self']

session.delete(base_url + assay_url)
session.delete(base_url + study_url)
session.delete(base_url + investigation_url)

"""Close the HTTP **session**"""

session.close()

# Exercise 5



*  Run the notebook using the user you created in the previous step
*  Update the attributes, and change the titles for the Investigation, Study and Assay.
*  Comment out the 3 delete steps at the end, and make a note of the resulting Assay ID. Check at https://sandbox9.fairdomhub.org/assays that it exists. 

